In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 500)
import plotly.graph_objects as go
import plotly.io as pio
pd.options.mode.copy_on_write = True
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
bac_original = pd.read_json("drive/MyDrive/bac_2024_full_sesiune_2.json")
bac_original

,nr,cod_candidat,unitate_invatamant,judet_cod,judet_nume,promotie_anterioara,forma_invatamant,specializare,limba_romana_competente,limba_romana_scris,limba_romana_contestatie,limba_romana_nota_finala,limba_materna,limba_materna_competente,limba_materna_scris,limba_materna_contestatie,limba_materna_nota_finala,limba_moderna,limba_moderna_nota,disciplina_obligatorie,disciplina_obligatorie_scris,disciplina_obligatorie_contestatie,disciplina_obligatorie_nota_finala,disciplina_aleasa,disciplina_aleasa_scris,disciplina_aleasa_contestatie,disciplina_aleasa_nota_finala,competente_digitale,media,rezultat_final
0,1,DB1500830,LICEUL TEORETIC ”PETRU CERCEL” TÂRGOVIȘTE,17,DB,False,Zi,Filologie,Utilizator experimentat,9.4,NaN,9.4,None,None,None,NaN,None,Limba engleză,B2-B1-B1-B2-B2,Istorie,9.6,NaN,9.6,Geografie,9.55,NaN,9.55,Utilizator nivel mediu,9.51,Reusit
1,2,IS1482406,"COLEGIUL NAȚIONAL DE ARTĂ ""OCTAV BĂNCILĂ"", IAŞI",26,IS,False,Zi,Muzica,Utilizator experimentat,9.1,NaN,9.1,None,None,None,NaN,None,Limba engleză,B1-B1-B1-B1-B1,Istorie,9.6,NaN,9.6,Geografie,9.8,NaN,9.8,Utilizator avansat,9.50,Reusit
2,3,CJ1491575,"LICEUL TEORETIC ""LUCIAN BLAGA"" CLUJ-NAPOCA",12,CJ,False,Zi,Științe ale Naturii,Utilizator experimentat,9.4,NaN,9.4,None,None,None,NaN,None,Limba engleză,B2-B2-B2-B2-B2,Matematică ST-NAT,9.5,NaN,9.5,"Anatomie și fiziologie umană, genetică și ecol...",9.5,NaN,9.5,Utilizator nivel mediu,9.46,Reusit
3,4,MS1526305,"LICEUL TEHNOLOGIC ""ION VLASIU"" TÎRGU MUREȘ",29,MS,False,Zi,Științe Sociale,Utilizator experimentat,8.9,NaN,8.9,None,None,None,NaN,None,Limba engleză,B2-B2-B2-B2-B2,Istorie,9.65,NaN,9.65,Geografie,9.75,NaN,9.75,Utilizator experimentat,9.43,Reusit
4,5,MH1521817,"COLEGIUL NAȚIONAL ""TRAIAN""",27,MH,False,Zi,Științe Sociale,Utilizator experimentat,8.8,NaN,8.8,None,None,None,NaN,None,Limba engleză,B2-B2-A2-B2-B2,Istorie,9.75,NaN,9.75,"Logică, argumentare și comunicare",9.55,NaN,9.55,Utilizator nivel mediu,9.36,Reusit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33675,33666,TR1451062,"LICEUL TEHNOLOGIC ""ANGHEL SALIGNY"" ROȘIORI DE ...",39,TR,False,Zi,Tehnician ecolog și protecția calității mediului,Utilizator avansat,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba engleză,A1-A2-A1-A2-A2,Matematică TEHN,Neprezentat,NaN,Neprezentat,Biologie vegetală și animală,Neprezentat,NaN,Neprezentat,Utilizator incepator,NaN,Eliminat din examen
33676,33667,TR1463742,"LICEUL TEHNOLOGIC ""VIRGIL MADGEARU"" ROȘIORI DE...",39,TR,False,Zi,Tehnician în activități economice,Utilizator avansat,5.15,NaN,5.15,None,None,None,NaN,None,Limba engleză,A1-A1-A1-A2-A2,Matematică TEHN,Eliminat din examen,NaN,Eliminat din examen,Geografie,7.15,NaN,7.15,Utilizator incepator,NaN,Eliminat din examen
33677,33668,TR1537398,"COLEGIUL NAȚIONAL ""ANASTASESCU"" ROȘIORI DE VEDE",39,TR,False,Zi,Filologie,Utilizator avansat,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba engleză,B1-A2-A1-A2-A2,Istorie,5.65,NaN,5.65,Geografie,8.0,NaN,8.0,Utilizator incepator,NaN,Eliminat din examen
33678,33669,TR1543161,LICEUL TEHNOLOGIC DRĂGĂNEȘTI-VLAȘCA,39,TR,False,Frecvență redusă,Matematica-Informatica,Utilizator experimentat,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba engleză,A2-A2-A1-A2-A2,Matematică MATE-INFO,Neprezentat,NaN,Neprezentat,Biologie vegetală și animală,Neprezentat,NaN,Neprezentat,Utilizator incepator,NaN,Eliminat din examen


In [ ]:
NEGATIVE_COLOR = "red"
POSITIVE_COLOR = "#2baaff"

In [ ]:
bac_for_mean = bac_original.copy()
bac_for_mean['limba_romana_nota_finala'] = pd.to_numeric(bac_for_mean['limba_romana_nota_finala'], errors='coerce')
bac_for_mean['disciplina_obligatorie_nota_finala'] = pd.to_numeric(bac_for_mean['disciplina_obligatorie_nota_finala'], errors='coerce')
bac_for_mean['disciplina_aleasa_nota_finala'] = pd.to_numeric(bac_for_mean['disciplina_aleasa_nota_finala'], errors='coerce')
bac_for_mean['limba_materna_nota_finala'] = pd.to_numeric(bac_for_mean['limba_materna_nota_finala'], errors='coerce')
bac_for_mean['media_calc'] = bac_for_mean[['limba_romana_nota_finala', 'disciplina_obligatorie_nota_finala', 'disciplina_aleasa_nota_finala', 'limba_materna_nota_finala']].mean(axis=1).apply(lambda x: x if x != x else math.trunc(x * 100) / 100)
bac_original["media_calc"] = bac_for_mean["media_calc"]

bac_for_mean_2 = bac_original.copy()
bac_for_mean_2['limba_romana_scris'] = pd.to_numeric(bac_for_mean_2['limba_romana_scris'], errors='coerce')
bac_for_mean_2['disciplina_obligatorie_scris'] = pd.to_numeric(bac_for_mean_2['disciplina_obligatorie_scris'], errors='coerce')
bac_for_mean_2['disciplina_aleasa_scris'] = pd.to_numeric(bac_for_mean_2['disciplina_aleasa_scris'], errors='coerce')
bac_for_mean_2['limba_materna_scris'] = pd.to_numeric(bac_for_mean_2['limba_materna_scris'], errors='coerce')
bac_for_mean_2['media_calc_original'] = bac_for_mean_2[['limba_romana_scris', 'disciplina_obligatorie_scris', 'disciplina_aleasa_scris', 'limba_materna_scris']].mean(axis=1).apply(lambda x: x if x != x else math.trunc(x * 100) / 100)
bac_original["media_calc_original"] = bac_for_mean_2["media_calc_original"]

In [ ]:
def val(bac, base_key, key_contestatie, key_final, title):
  df = pd.DataFrame()
  df["base"] = pd.to_numeric(bac[base_key], errors="coerce")
  df["contestatie"] = pd.to_numeric(bac[key_contestatie], errors="coerce")
  df["final"] = pd.to_numeric(bac[key_final], errors="coerce")

  df.dropna(subset=["base"], inplace=True)
  df["title"] = "Disciplina " + title
  return df

def sort_by_count(x):
  return sorted(x, key=lambda a: a["contestatie"].count(), reverse=True)

VAL_ROMANA = val(bac_original, "limba_romana_scris", "limba_romana_contestatie", "limba_romana_nota_finala", "Limba și literatura română")
VALS_OBLIGATORII = []
VALS_OBLIGATORII_ALL = []
for obligatorie in bac_original["disciplina_obligatorie"].unique().tolist():
  filtered = bac_original[bac_original["disciplina_obligatorie"] == obligatorie]
  data = val(filtered, "disciplina_obligatorie_scris", "disciplina_obligatorie_contestatie", "disciplina_obligatorie_nota_finala", obligatorie)
  if data["contestatie"].count() > 1000:
    VALS_OBLIGATORII.append(data)
  VALS_OBLIGATORII_ALL.append(data)

VALS_ALESE = []
VALS_ALESE_ALL = []
for aleasa in bac_original["disciplina_aleasa"].unique().tolist():
  filtered = bac_original[bac_original["disciplina_aleasa"] == aleasa]
  data = val(filtered, "disciplina_aleasa_scris", "disciplina_aleasa_contestatie", "disciplina_aleasa_nota_finala", aleasa)
  if data["contestatie"].count() > 1000:
    VALS_ALESE.append(data)
  VALS_ALESE_ALL.append(data)

VALS_OBLIGATORII = sort_by_count(VALS_OBLIGATORII)
VALS_OBLIGATORII_ALL = sort_by_count(VALS_OBLIGATORII_ALL)

VALS_ALESE = sort_by_count(VALS_ALESE)
VALS_ALESE_ALL = sort_by_count(VALS_ALESE_ALL)

VALS_ALL = [VAL_ROMANA, *VALS_OBLIGATORII_ALL, *VALS_ALESE_ALL]
VALS_SELECTED = [VAL_ROMANA, *VALS_OBLIGATORII, *VALS_ALESE]

In [ ]:
def final_results_graph(bac):
  passing_hist = go.Histogram(x=bac[bac["rezultat_final"] == "Reusit"]["media"], name="Candidați admiși")
  failing_hist = go.Histogram(x=bac[bac["rezultat_final"] == "Respins"]["media_calc"], name="Candidați respinși")

  fig = go.Figure(layout_title_text="Media finală")
  fig.add_trace(passing_hist)
  fig.add_trace(failing_hist)
  fig.update_layout(barmode="overlay", xaxis_title="Media probelor date", yaxis_title="Număr candidați")
  return fig

In [ ]:
RESULTS_BY_DISCIPLINE_GRAPHS = []
def results_by_discipline_graph(bac, title, percent):
  num_students = len(bac.index)

  counts = bac["final"].value_counts().sort_index().reset_index()
  counts.columns = ["final", 'count']

  fig_line = go.Scatter(x=counts["final"],
                        y=counts["count"].divide(num_students / 100 if percent else 1),
                        mode="lines",
                        name=title,
                        # line_shape='spline',
                        # line_smoothing=1.3
                    )

  return fig_line

for graph_input in [
    ["Limba și literatura română", [VAL_ROMANA]],
    ["Disciplina obligatorie profilului", VALS_OBLIGATORII_ALL, VALS_OBLIGATORII],
    ["Disciplina aleasă", VALS_ALESE_ALL, VALS_ALESE]
  ]:
  fig_nr_medii = go.Figure(layout_title_text=f"Nota finală - {graph_input[0]}")
  fig_nr_medii.update_layout(xaxis_title="Nota finală (după contestații)", yaxis_title="Număr de note")
  fig_nr_medii.update_layout(legend_title_text=graph_input[0])
  for bac in graph_input[1]:
    fig_nr_medii.add_trace(results_by_discipline_graph(bac, bac["title"].iloc[0], False))
  fig_nr_medii.show()
  RESULTS_BY_DISCIPLINE_GRAPHS.append(fig_nr_medii)

  if len(graph_input) == 3:
    fig_nr_medii_percent = go.Figure(layout_title_text=f"Nota finală - {graph_input[0]} - Procent din candidați")
    fig_nr_medii_percent.update_layout(xaxis_title="Nota finală (după contestații)", yaxis_title="Procent din notele disciplinei")
    fig_nr_medii_percent.update_layout(legend_title_text=graph_input[0])
    for bac in graph_input[2]:
      fig_nr_medii_percent.add_trace(results_by_discipline_graph(bac, bac["title"].iloc[0], True))
    fig_nr_medii_percent.update_yaxes(ticksuffix="%")
    fig_nr_medii_percent.show()
    RESULTS_BY_DISCIPLINE_GRAPHS.append(fig_nr_medii_percent)

In [ ]:
POINTS_GAINED_GRAPHS = []
for bac in VALS_SELECTED:
  bac = bac.dropna()
  bac['difference'] = bac["contestatie"] - bac["base"]

  points = go.Scatter(x=bac["base"], y=bac["difference"],
                   marker=dict(size=7, opacity=0.3, color=bac["difference"], cmin=-0.1, cmax=0.1, colorscale=[[0, NEGATIVE_COLOR], [0.5, "white"], [1, POSITIVE_COLOR]]),
                      mode="markers"
  )

  fig = go.Figure(layout_title_text=f"Punctele dobândite din contestație - {bac['title'].iloc[0]}")
  fig.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Δ = Nota contestată - Nota inițială")
  fig.add_trace(points)
  fig.update_yaxes(ticksuffix=" puncte")
  # fig.show()
  POINTS_GAINED_GRAPHS.append(fig)
POINTS_GAINED_GRAPHS[0].show()

In [ ]:
DELTA_GRAPHS = []
for bac in VALS_SELECTED:
  bac = bac.dropna()
  bac['difference'] = bac["contestatie"] - bac["base"]

  difference_positive = bac[bac["difference"] > 0]
  difference_negative = bac[bac["difference"] < 0]
  difference_same = bac[bac["difference"] == 0]

  counts_positive = difference_positive["base"].value_counts().sort_index().reset_index()
  counts_positive.columns = ["base", 'count']

  counts_negative = difference_negative["base"].value_counts().sort_index().reset_index()
  counts_negative.columns = ["base", 'count']

  counts_same = difference_same["base"].value_counts().sort_index().reset_index()
  counts_same.columns = ["base", "count"]

  positive_delta = go.Bar(
      x=counts_positive["base"],
      y=counts_positive['count'],
      name='Nr. note mai mari după contestație',
      marker=dict(color=POSITIVE_COLOR)
  )
  negative_delta = go.Bar(
      x=counts_negative["base"],
      y=counts_negative['count'],
      name='Nr. note mai mici după contestație',
      marker=dict(color=NEGATIVE_COLOR)
  )
  neutral_delta = go.Bar(
      x=counts_same["base"],
      y=counts_same['count'],
      name='Nr. note la fel după contestație',
      marker=dict(color='gray'),
  )

  fig = go.Figure(layout_title_text=f"Schimbarea notei după contestație - {bac['title'].iloc[0]}")
  fig.add_trace(positive_delta)
  fig.add_trace(negative_delta)
  fig.add_trace(neutral_delta)

  fig.update_layout(xaxis_title="Nota înainte de contestație")
  fig.update_layout(legend_title_text='Efectul contestației')
  fig.update_xaxes(nticks=10, showgrid=True)
  fig.update_yaxes(nticks=10, showgrid=True)

  fig.update_traces(width=0.04)

  fig.update_layout(xaxis=dict(range=[1,10]))
  # fig.show()
  DELTA_GRAPHS.append(fig)
DELTA_GRAPHS[0].show()

In [58]:
COUNT_GRAPHS = []
def graph_line(bac, title, percent):
  num_students = len(bac.index)

  counts = bac["base"].value_counts().sort_index().reset_index()
  counts.columns = ["base", 'count']

  fig_line = go.Scatter(x=counts["base"],
                        y=counts["count"].divide(num_students / 100 if percent else 1),
                        mode="lines",
                        name=title,
                    )

  return fig_line

for graph_input in [
    ["Limba și literatura română", [VAL_ROMANA]],
    ["Disciplina obligatorie profilului", VALS_OBLIGATORII_ALL, VALS_OBLIGATORII],
    ["Disciplina aleasă", VALS_ALESE_ALL, VALS_ALESE]
  ]:
  fig_nr_contestatii = go.Figure(layout_title_text=f"Număr de contestații versus nota inițială - {graph_input[0]}")
  fig_nr_contestatii.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Număr de contestații")
  fig_nr_contestatii.update_layout(legend_title_text=graph_input[0])
  for bac in graph_input[1]:
    bac = bac.dropna()
    if bac.shape[0] == 0:
      continue
    fig_nr_contestatii.add_trace(graph_line(bac, bac["title"].iloc[0], False))
  fig_nr_contestatii.show()
  COUNT_GRAPHS.append(fig_nr_contestatii)

  if len(graph_input) == 3:
    fig_nr_contestatii_percent = go.Figure(layout_title_text=f"Număr de contestații versus nota inițială - {graph_input[0]} - Procent din candidați")
    fig_nr_contestatii_percent.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Procent din candidații disciplinei")
    fig_nr_contestatii_percent.update_layout(legend_title_text=graph_input[0])
    for bac in graph_input[2]:
      bac = bac.dropna()
      fig_nr_contestatii_percent.add_trace(graph_line(bac, bac["title"].iloc[0], True))
    fig_nr_contestatii_percent.update_yaxes(ticksuffix="%")
    fig_nr_contestatii_percent.show()
    COUNT_GRAPHS.append(fig_nr_contestatii_percent)


In [ ]:
bac_original[bac_original["rezultat_final"] == "Respins"][bac_original["media_calc"] >= 6].sort_values(by=["media_calc"], ascending=False)

<ipython-input-48-f2ec311c0558>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,nr,cod_candidat,unitate_invatamant,judet_cod,judet_nume,promotie_anterioara,forma_invatamant,specializare,limba_romana_competente,limba_romana_scris,limba_romana_contestatie,limba_romana_nota_finala,limba_materna,limba_materna_competente,limba_materna_scris,limba_materna_contestatie,limba_materna_nota_finala,limba_moderna,limba_moderna_nota,disciplina_obligatorie,disciplina_obligatorie_scris,disciplina_obligatorie_contestatie,disciplina_obligatorie_nota_finala,disciplina_aleasa,disciplina_aleasa_scris,disciplina_aleasa_contestatie,disciplina_aleasa_nota_finala,competente_digitale,media,rezultat_final,media_calc,media_calc_original
21752,21743,HR1490876,"COLEGIUL TEHNIC ""BATTHYÁNY IGNÁC"" GHEORGHENI",23,HR,False,Zi,Tehnician în industria alimentară,Utilizator avansat,3.6,4.50,4.5,Limba maghiară (REAL),Utilizator experimentat,7.45,NaN,7.45,Limba engleză,A2-A2-A2-A2-B1,Matematică TEHN,9.85,NaN,9.85,"Anatomie și fiziologie umană, genetică și ecol...",8.95,NaN,8.95,Utilizator nivel mediu,NaN,Respins,7.68,7.46
19323,19314,CV1515289,"Liceul Teoretic ""Mikes Kelemen"" Sfântu Gheorghe",16,CV,False,Zi,Științe Sociale,Utilizator experimentat,3.85,4.20,4.2,Limba maghiară (UMAN),Utilizator experimentat,8.25,NaN,8.25,Limba engleză,A2-A2-A1-A2-A2,Istorie,9.05,NaN,9.05,"Logică, argumentare și comunicare",9.05,NaN,9.05,Utilizator nivel mediu,NaN,Respins,7.63,7.55
21719,21710,HR1473026,"LICEUL TEHNOLOGIC ""KÓS KÁROLY"" ODORHEIU SECUIESC",23,HR,False,Zi,Tehnician în turism,Utilizator avansat,3.7,3.80,3.8,Limba maghiară (REAL),Utilizator experimentat,8.3,NaN,8.3,Limba engleză,A2-A2-A1-A2-A2,Matematică TEHN,9.2,NaN,9.2,Geografie,8.95,NaN,8.95,Utilizator nivel mediu,NaN,Respins,7.56,7.53
27247,27238,VL1234319,"LICEUL PREDA BUZESCU, ORAS BERBESTI",40,VL,True,Zi,Tehnician mecatronist,Utilizator nivel mediu,4.7,NaN,4.7,None,None,None,NaN,None,Limba engleză,A2-B1--A1-A1,Matematică TEHN,7.9,NaN,7.9,Biologie vegetală și animală,10.0,NaN,10.0,None,NaN,Respins,7.53,7.53
14249,14240,AR1427441,LICEUL TEORETIC PÂNCOTA,3,AR,True,Zi,Matematica-Informatica,Utilizator avansat,9.0,NaN,9.0,None,None,None,NaN,None,Limba engleză,A2-A2-A2-A2-A2,Matematică MATE-INFO,4.65,4.60,4.6,"Anatomie și fiziologie umană, genetică și ecol...",8.65,NaN,8.65,Utilizator experimentat,NaN,Respins,7.41,7.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23339,23330,MM1434071,"COLEGIUL TEHNIC ""GEORGE BARITIU"" BAIA MARE",28,MM,True,Seral,Tehnician operator tehnică de calcul,Utilizator nivel mediu,7.2,NaN,7.2,None,None,None,NaN,None,Limba engleză,B1-B1-B1-B1-B1,Matematică TEHN,4.75,4.60,4.6,Biologie vegetală și animală,6.2,NaN,6.2,Utilizator experimentat,NaN,Respins,6.00,6.05
21853,21844,HR1524095,"COLEGIUL TEHNIC ""BATTHYÁNY IGNÁC"" GHEORGHENI",23,HR,False,Zi,Tehnician în activități economice,Utilizator avansat,4.35,4.15,4.15,Limba maghiară (REAL),Utilizator experimentat,7.0,NaN,7.0,Limba engleză,A2-A2-A2-A2-A2,Matematică TEHN,5.7,NaN,5.7,Geografie,7.15,NaN,7.15,Utilizator incepator,NaN,Respins,6.00,6.05
19457,19448,DB1347379,LICEUL TEORETIC ”IANCU C VISSARION” TITU,17,DB,True,Zi,Matematica-Informatica,Utilizator nivel mediu,6.7,NaN,6.7,None,None,None,NaN,None,Limba franceză,A1-A1-A1-A1-A1,Matematică MATE-INFO,4.45,4.65,4.65,"Anatomie și fiziologie umană, genetică și ecol...",6.65,NaN,6.65,Utilizator avansat,NaN,Respins,6.00,5.93
22118,22109,IF1511676,"LICEUL TEHNOLOGIC ""PAMFIL ȘEICARU"" CIOROGÂRLA",24,IF,False,Zi,Tehnician în activități economice,Utilizator experimentat,5.4,NaN,5.4,None,None,None,NaN,None,Limba engleză,B2-B1-A1-A2-A2,Matematică TEHN,4.4,4.75,4.75,Geografie,7.85,NaN,7.85,Utilizator nivel mediu,NaN,Respins,6.00,5.88


In [ ]:
# Elevi admiși dar respinși după contestații
to_filter = bac_original[bac_original["rezultat_final"] == "Respins"][bac_original["media_calc_original"] >= 6][bac_original["media_calc"] < 6]
failed_after_contestatie = []
for index, row in to_filter.iterrows():
  if row["limba_romana_scris"] is not None and row["limba_romana_scris"] < 5:
    continue
  if row["disciplina_obligatorie_scris"] is not None and row["disciplina_obligatorie_scris"] < 5:
    continue
  if row["disciplina_aleasa_scris"] is not None and row["disciplina_aleasa_scris"] < 5:
    continue
  if row["limba_materna_scris"] is not None and row["limba_materna_scris"] < 5:
    continue
  failed_after_contestatie.append(row)

<ipython-input-49-6403e5f9c069>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-49-6403e5f9c069>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
VAL_ROMANA[VAL_ROMANA["base"] > 6.5][VAL_ROMANA["base"] < 9][VAL_ROMANA["final"] - VAL_ROMANA["base"] > 0]

<ipython-input-50-fb5ab1a80fe4>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-50-fb5ab1a80fe4>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,base,contestatie,final,title
27,8.95,9.50,9.50,Disciplina Limba și literatura română
49,7.75,8.45,8.45,Disciplina Limba și literatura română
105,7.45,7.90,7.90,Disciplina Limba și literatura română
176,6.65,7.90,7.90,Disciplina Limba și literatura română
5472,6.90,7.80,7.80,Disciplina Limba și literatura română
6070,6.95,7.75,7.75,Disciplina Limba și literatura română
7133,6.80,7.70,7.70,Disciplina Limba și literatura română
7642,6.80,7.20,7.20,Disciplina Limba și literatura română
7794,6.60,7.70,7.70,Disciplina Limba și literatura română
8041,6.60,6.95,6.95,Disciplina Limba și literatura română


In [ ]:
failed_with_one_max_score = bac_original[bac_original["rezultat_final"] == "Respins"][(bac_original["disciplina_aleasa_nota_finala"] == 10) | (bac_original["limba_romana_nota_finala"] == 10) | (bac_original["limba_materna_nota_finala"] == 10) | (bac_original["disciplina_obligatorie_nota_finala"] == 10)]
failed_with_one_max_score

<ipython-input-53-e61b01dd38c9>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,nr,cod_candidat,unitate_invatamant,judet_cod,judet_nume,promotie_anterioara,forma_invatamant,specializare,limba_romana_competente,limba_romana_scris,limba_romana_contestatie,limba_romana_nota_finala,limba_materna,limba_materna_competente,limba_materna_scris,limba_materna_contestatie,limba_materna_nota_finala,limba_moderna,limba_moderna_nota,disciplina_obligatorie,disciplina_obligatorie_scris,disciplina_obligatorie_contestatie,disciplina_obligatorie_nota_finala,disciplina_aleasa,disciplina_aleasa_scris,disciplina_aleasa_contestatie,disciplina_aleasa_nota_finala,competente_digitale,media,rezultat_final,media_calc,media_calc_original
24290,24281,NT1300867,"LICEUL TEHNOLOGIC ""ION IONESCU DE LA BRAD"", CO...",30,NT,True,Seral,Tehnician agronom,Utilizator avansat,2.2,3.3,3.3,None,None,None,NaN,None,Limba engleză,-A2--A2-B1,Matematică TEHN,10.0,NaN,10.0,"Anatomie și fiziologie umană, genetică și ecol...",4.3,3.95,3.95,None,NaN,Respins,5.75,5.50
27247,27238,VL1234319,"LICEUL PREDA BUZESCU, ORAS BERBESTI",40,VL,True,Zi,Tehnician mecatronist,Utilizator nivel mediu,4.7,NaN,4.7,None,None,None,NaN,None,Limba engleză,A2-B1--A1-A1,Matematică TEHN,7.9,NaN,7.9,Biologie vegetală și animală,10.0,NaN,10.0,None,NaN,Respins,7.53,7.53


In [ ]:
!pip install airium

In [ ]:
categories_list = []
def include_plots_by_category(plots, category, comment=None):
  print(f"Adding plots for category {category}")
  html = ""
  for idx, plot in enumerate(plots):
    html += pio.to_html(plot, include_plotlyjs=False, full_html=False)
  categories_list.append(dict(category=category, html=html, comment=comment))

include_plots_by_category([final_results_graph(bac_original)], "Media finală", ["Un candidat poate fi respins, chiar dacă are media finală mai mare sau egală decât 6, în cazul în care una sau mai multe din probele date au nota sub 5."])
include_plots_by_category(RESULTS_BY_DISCIPLINE_GRAPHS, "Notele la fiecare disciplină", ["Se observă anumite rotunjiri, întrucât sunt multe note egale cu 5,00."])
include_plots_by_category(COUNT_GRAPHS, "Număr de contestații versus nota inițială")
include_plots_by_category(DELTA_GRAPHS, "Schimbarea notei după contestație", ["O contestație la Limba și literatura română cu nota între 6,5 și 9 avea o șansă de ~69% să fie favorabilă (să rezulte într-o notă finală mai mare)."])
include_plots_by_category(POINTS_GAINED_GRAPHS, "Punctele dobândite din contestație")

Adding plots for category Media finală
Adding plots for category Notele la fiecare disciplină
Adding plots for category Număr de contestații versus nota inițială
Adding plots for category Schimbarea notei după contestație
Adding plots for category Punctele dobândite din contestație


In [ ]:

from airium import Airium
a = Airium()
def link_candidate(x):
  a.a(_t=x["cod_candidat"], href=f"http://bacalaureat.edu.ro/RezultateCautare.aspx?Jud={x['judet_cod']}&Cod={x['cod_candidat']}", style="color:red")

a('<!DOCTYPE html>')
with a.html(lang="ro"):
    with a.head():
        a.meta(charset="utf-8")
        a.title(_t="Statistici Bac 2024 (august)")
        a.meta(property="og:title", content="Statistici Bac 2024 (august)")
        a.meta(property="og:description", content="Câteva statistici despre rezultatele Bacalaureatului din 2024")
        a.script(src="https://cdn.plot.ly/plotly-2.33.0.min.js", charset="utf-8")

    with a.body():
      a.h1(_t="Statistici Bac 2024 (august)")
      with a.p():
        a("Datele au fost obținute din site-ul bacalaureat.edu.ro și pot fi descărcate aici: ")
        a.a(_t="https://github.com/tacheometry/rezultate-bac-2024", href="https://github.com/tacheometry/rezultate-bac-2024")
      with a.p():
        a("Pentru sugestii de mai multe vizualizări/etc ")
        a.a(_t="trimiteți un Issue pe GitHub", href="https://github.com/tacheometry/rezultate-bac-2024/issues/new")
        a(".")

      a.h2(_t="Cuprins")
      with a.ol():
        for category in [*categories_list, dict(category="Condoleanțe")]:
          with a.li():
            a.a(_t=category["category"], href=f"#{category['category']}")

      for category in categories_list:
          a.h2(_t=category["category"], id=category["category"])
          if "comment" in category and category["comment"] is not None:
            with a.ul():
              for subcomment in category["comment"]:
                a.li(_t=subcomment)
          a(category["html"])
      a.h2(_t="Condoleanțe", id="Condoleanțe")
      a.h3(_t="Candidați admiși inițial dar respinși după contestații")
      with a.ol():
        for x in failed_after_contestatie:
          with a.li():
            link_candidate(x)
      a.h3(_t="Candidați cu o notă de 10 respinși")
      with a.ol():
        for idx, x in failed_with_one_max_score.iterrows():
          with a.li():
            link_candidate(x)

with open('rezultate_bac_2024_august.html', "wb") as f:
    f.write(bytes(a))